# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [8]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']
print("ok")

ok


# Making random queries to the API

In [9]:
challenge_id = 'RunxingJia78911' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
print(queries)
query_results = query(challenge_id, queries)

print(query_results)


[[-1  1  1 ...  1 -1  1]
 [-1 -1 -1 ...  1 -1  1]
 [-1 -1  1 ... -1  1 -1]
 ...
 [ 1  1  1 ... -1 -1  1]
 [-1 -1 -1 ... -1  1 -1]
 [-1  1  1 ... -1  1  1]]
[ -7. -12.  -6.   1.  -9.  -0.  -9.  -2.   3.   7.  -7.  16. -18.  -7.
 -14.  22.   6.  21.  17.  -9.  11. -15.  24.  22. -26.   7. -34. -19.
  -4. -15.  10.  14.  -7. -12. -15.  17.  12. -20.   1. -10. -25.   9.
  13. -65. -18.  10.   3.   7.   7.  -3. -11. -16.  10.  -3. -12.   5.
  11. -45.  -2.   3.  -1.  -9.  28.  -8.  41.  27.  36.   3. -13.  11.
  -2.   5.  33.   4.   4.  12.  20. -11.   9.   0.  -3. -12.  -9.   9.
   9.   0.  17. -36.   2. -24.   4.  19.  -6. -40. -14.  -0.   4.   2.
  15. -17.   2.   1.   9.   7.   9.  -9. -16.  20.   2. -12.  -1.  -6.
 -11.  24.   7.   6. -13.  -3.  -4. -18.   9.  15.  11.  -3.  -3.   7.
 -21.  -1.   7. -13.  -2.  -2. -10. -14.   4.  17.  19. -12. -24.  10.
  -7. -30.  12.   5.   8.  -1. -14. -23. -22.   2.  -5.  -9.   7. -39.
  -2.   5.  11.  -6.   3.  -3.  24.  33.   6.  -0.  -6.  -4.  6

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [10]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.61 correct values


In [11]:
import numpy as np
from scipy.optimize import linprog

n = 100        
t = 2 * n       
lambda_ = 10    
challenge_id = 'RunxingJia78911'


queries = np.random.choice([-1, 1], size=(t, n))
print("queries:",queries.shape)


query_results = query(challenge_id, queries)
print("query_results:",query_results.shape)

num_vars = n + t 

#Coefficients of the objective function
c = np.hstack([np.zeros(n), np.ones(t)])

print("c:",c.shape)
A_ub = []
b_ub = []

for j in range(t):
    q = queries[j]
    y = query_results[j]
    
    # q_j, x⟩ - y_j - e_j ≤ 0
    A_row = np.hstack([q, np.zeros(t)])
    A_row[n + j] = -1  #  -e_j
    A_ub.append(A_row)
    b_ub.append(y)
    
    # -⟨q_j, x⟩ + y_j - e_j ≤ 0
    A_row = np.hstack([-q, np.zeros(t)])
    A_row[n + j] = -1  #  -e_j
    A_ub.append(A_row)
    b_ub.append(-y)

A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

print("A_ub:",A_ub.shape)
print("b_ub:",b_ub.shape)

bounds = [(-1, 1)] * n + [(0, None)] * t

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print(result.x.shape)

if result.success:
    x_estimated = result.x[:n]
    #  -1 or 1
    x_reconstructed = np.sign(x_estimated)
    
else:
    print("fail!!!")


print(f"Reconstructed x: {x_reconstructed}")
best_query_result = query(challenge_id, x_reconstructed, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")



queries: (200, 100)
query_results: (200,)
c: (300,)
A_ub: (400, 300)
b_ub: (400,)
(300,)
Reconstructed x: [ 1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1. -1. -1.  1.
  1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1.
  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1. -1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.  1.  1.
  1.  1.  1.  1. -1. -1.  1.  1. -1. -1.]

Reconstruction attack achieves fraction 0.94 correct values
